# Omission LFP Analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import glob
from collections import defaultdict
import re

In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"
import cupy as cp

In [4]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
import itertools
from scipy.stats import linregress
import cupy as cp

In [5]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [6]:
from statsmodels.tsa.stattools import grangercausalitytests


## Inputs & Data

Explanation of each input and where it comes from.

In [7]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5


In [8]:
FREQ_MAX = 100

In [9]:
# Define your keywords
KEYWORD_TOP = '_trial_'
KEYWORD_BOTTOM = '_baseline_'

In [10]:
GROUPING = "all"
# GROUPING = "velocity_parsed"

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

In [12]:
BBOX_TO_ANCHOR=(1.5, 0.9)
LOC='upper right'

In [13]:
ALL_BANDS = ["theta", "beta", "gamma"]
BAND_TO_FREQ = {"theta": {"low_freq": 4, "high_freq": 12}, "beta": {"low_freq": 13, "high_freq": 30}, "gamma": {"low_freq": 30, "high_freq": 70}}

In [14]:
# variables for LFP extraction
FREQ_MIN=0.5
FREQ_MAX=300
NOTCH_FREQ=60
ORIGINAL_SAMPLE_RATE = 20000
RESAMPLE_RATE=1000
TRIAL_DURATION=10

In [15]:
BASELINE_OUTCOME_TO_COLOR = {
'lose': "red",
'lose_trial': "red",
'lose_baseline': "hotpink",

'omission': "orange",
'omission_trial': "orange",
'omission_baseline': "navajowhite",

'rewarded': "green",
'rewarded_trial': "green",
'rewarded_baseline': "lightgreen",

'win': "blue",
'win_trial': "blue",
'win_baseline': "lightblue"}

In [16]:
CHANNEL_MAPPING_DF = pd.read_excel("../../data/channel_mapping.xlsx")
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

TONE_TIMESTAMP_DF = pd.read_excel("../../data/rce_tone_timestamp.xlsx", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs


In [17]:
def standardize_col_name(col_name, keywords, replacement="_"):
    """
    Standardize the column name by removing specified keywords.

    Parameters:
    col_name (str): The original column name.
    keywords (list of str): A list of keywords to remove from the column name.

    Returns:
    str: The standardized column name.
    """
    for keyword in keywords:
        col_name = col_name.replace(keyword, replacement)
    # Replace any double underscores possibly created and strip leading/trailing underscores
    col_name = col_name.replace('__', '_').strip('_')
    return col_name

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions

- Ideally functions are defined here first and then data is processed using the functions
    - function names are short and in snake case all lowercase
    - a function name should be unique but does not have to describe the function
    - doc strings describe functions not function names

In [18]:
def generate_pairs(lst):
    pairs = []
    n = len(lst)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((lst[i], lst[j]))
    return pairs

In [19]:
def nested_dict():
    return defaultdict(dict)

triple_nested_dict = defaultdict(nested_dict)

## Processing

In [20]:
BASIC_LFP_TRACES_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")

In [21]:
BASIC_LFP_TRACES_DF = BASIC_LFP_TRACES_DF.drop_duplicates(["recording_file", "time", "current_subject"])

In [22]:
BASIC_LFP_TRACES_DF.columns

Index(['time', 'recording_dir', 'recording_file', 'time_stamp_index',
       'video_file', 'video_frame', 'video_number', 'subject_info',
       'competition_closeness', 'video_name', 'all_subjects',
       'current_subject', 'trial_outcome', 'lfp_index',
       'baseline_lfp_timestamp_range', 'trial_lfp_timestamp_range',
       'baseline_ephys_timestamp_range', 'trial_ephys_timestamp_range',
       'baseline_videoframe_range', 'trial_videoframe_range', 'trial_number',
       'Cohort', 'spike_interface_mPFC', 'spike_interface_vHPC',
       'spike_interface_BLA', 'spike_interface_LH', 'spike_interface_MD',
       'mPFC_baseline_lfp_trace', 'mPFC_trial_lfp_trace',
       'vHPC_baseline_lfp_trace', 'vHPC_trial_lfp_trace',
       'BLA_baseline_lfp_trace', 'BLA_trial_lfp_trace',
       'LH_baseline_lfp_trace', 'LH_trial_lfp_trace', 'MD_baseline_lfp_trace',
       'MD_trial_lfp_trace'],
      dtype='object')

In [23]:
BASIC_LFP_TRACES_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,...,mPFC_baseline_lfp_trace,mPFC_trial_lfp_trace,vHPC_baseline_lfp_trace,vHPC_trial_lfp_trace,BLA_baseline_lfp_trace,BLA_trial_lfp_trace,LH_baseline_lfp_trace,LH_trial_lfp_trace,MD_baseline_lfp_trace,MD_trial_lfp_trace
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,"[1.8457601, 1.7363818, 1.6475118, 1.59738, 1.2...","[0.6927297, 0.96389693, 0.7884358, -0.04101689...","[-0.06969439, -0.09568214, -0.05315674, 0.1571...","[1.5864334, 1.5710771, 1.5970649, 1.2155175, 0...","[2.0367627, 2.1163385, 2.1618104, 2.2679114, 2...","[0.3164087, 0.36377528, 0.18757163, -0.5020857...","[3.1382985, 3.2319791, 3.2788196, 3.2881875, 3...","[0.8118982, 1.2209699, 0.87435186, -0.4028264,...","[1.3934726, 1.494771, 1.764077, 1.828315, 1.68...","[-0.9783956, -0.86721426, -0.7288553, -1.40582..."
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,"[1.2191132, 1.1348007, 1.2054409, 1.0960625, 0...","[1.0732753, 0.7246318, 0.7633699, 0.3782669, -...","[0.31539667, 0.23152715, 0.29767776, 0.4217101...","[0.03543783, -0.27641505, -0.40044746, -0.6638...","[0.3107247, 0.14209972, -0.05873455, -0.331566...","[0.026525281, -0.04547191, 0.11936376, -0.4092...","[-1.180375, -1.2959143, -1.3771042, -1.458294,...","[0.9492963, 0.46840277, 0.6713773, 0.043717593...","[-0.14577106, -0.16059524, 0.027177656, 0.1680...","[1.6281886, 1.349, 1.4675934, 0.9487473, -0.21..."
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,"[-1.2669662, -1.2965895, -1.2532939, -0.986684...","[0.28711826, 0.84996116, 1.0960625, 0.8226166,...","[-1.2556804, -1.2580429, -1.3312811, -1.118654...","[0.060244307, 0.4748669, 0.7654571, 0.6591436,...","[-1.9912907, -1.9041362, -1.9325562, -1.542255...","[0.69344664, 1.4001559, 1.7582471, 1.4304705, ...","[-0.19985186, -0.074944444, -0.18423842, -0.13...","[-0.59643286, 0.27167362, 0.6901134, 0.4371759...","[-0.32119048, -0.52872896, -0.96851283, -0.753...","[0.096357144, 0.88450915, 1.2131118, 0.8943919..."
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,20221202_134600_omission_and_competition_subje...,...,"[-2.0257788, -2.0348935, -1.9323514, -1.754611...","[2.376701, 2.3015034, 1.7796774, 0.9411098, 0....","[0.16655779, 0.42879772, 0.66268736, 0.6934002...","[-1.8427671, -2.303459, -2.6802812, -3.060647,...","[-1.2637402, -1.0382752, -0.82986236, -0.74649...","[2.6771586, 2.3929594, 2.209177, 1.9761335, 1....","[-2.538743, -2.1983705, -1.8673657, -1.7143542...","[2.8447661, 2.3045416, 1.5301157, 0.96490973, ...","[-2.7647088, -2.5546997, -2.3051593, -2.055619...","[2.087738, 1.7418406, 1.1266373, 0.45954946, 0..."
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,...,"[-0.5765152, 0.25749493, 0.6403192, 0.4375135,...","[-0.043295607, 0.73602533, 0.31674156, 0.07747...","[-0.31421542, 0.19727057, 0.4453354, 0.3744597...","[0.21617076, 0.8221576, 0.58236164, 0.43116024...","[-2.1352851, -2.0576038, -2.0822346, -2.140969...","[-0.18188764, 0.113

In [24]:
trace_columns = [col for col in BASIC_LFP_TRACES_DF.columns if "trace" in col]

In [25]:
id_cols = [col for col in BASIC_LFP_TRACES_DF.columns if col not in trace_columns]

In [26]:
trace_columns

['mPFC_baseline_lfp_trace',
 'mPFC_trial_lfp_trace',
 'vHPC_baseline_lfp_trace',
 'vHPC_trial_lfp_trace',
 'BLA_baseline_lfp_trace',
 'BLA_trial_lfp_trace',
 'LH_baseline_lfp_trace',
 'LH_trial_lfp_trace',
 'MD_baseline_lfp_trace',
 'MD_trial_lfp_trace']

In [27]:
BASIC_LFP_TRACES_DF = BASIC_LFP_TRACES_DF.reset_index()

## Coherece Calculation

- Getting the brain region pairs

In [28]:
trial_trace_columns = sorted([col for col in BASIC_LFP_TRACES_DF.columns if "trial_lfp_trace" in col])
baseline_trace_columns = sorted([col for col in BASIC_LFP_TRACES_DF.columns if "baseline_lfp_trace" in col])

In [29]:
trial_trace_columns

['BLA_trial_lfp_trace',
 'LH_trial_lfp_trace',
 'MD_trial_lfp_trace',
 'mPFC_trial_lfp_trace',
 'vHPC_trial_lfp_trace']

In [30]:
trial_brain_region_pairs = generate_pairs(trial_trace_columns)
trial_brain_region_pairs = sorted(trial_brain_region_pairs)
baseline_brain_region_pairs = generate_pairs(baseline_trace_columns)
baseline_brain_region_pairs = sorted(baseline_brain_region_pairs)

In [31]:
trial_brain_region_pairs

[('BLA_trial_lfp_trace', 'LH_trial_lfp_trace'),
 ('BLA_trial_lfp_trace', 'MD_trial_lfp_trace'),
 ('BLA_trial_lfp_trace', 'mPFC_trial_lfp_trace'),
 ('BLA_trial_lfp_trace', 'vHPC_trial_lfp_trace'),
 ('LH_trial_lfp_trace', 'MD_trial_lfp_trace'),
 ('LH_trial_lfp_trace', 'mPFC_trial_lfp_trace'),
 ('LH_trial_lfp_trace', 'vHPC_trial_lfp_trace'),
 ('MD_trial_lfp_trace', 'mPFC_trial_lfp_trace'),
 ('MD_trial_lfp_trace', 'vHPC_trial_lfp_trace'),
 ('mPFC_trial_lfp_trace', 'vHPC_trial_lfp_trace')]

In [32]:
baseline_brain_region_pairs

[('BLA_baseline_lfp_trace', 'LH_baseline_lfp_trace'),
 ('BLA_baseline_lfp_trace', 'MD_baseline_lfp_trace'),
 ('BLA_baseline_lfp_trace', 'mPFC_baseline_lfp_trace'),
 ('BLA_baseline_lfp_trace', 'vHPC_baseline_lfp_trace'),
 ('LH_baseline_lfp_trace', 'MD_baseline_lfp_trace'),
 ('LH_baseline_lfp_trace', 'mPFC_baseline_lfp_trace'),
 ('LH_baseline_lfp_trace', 'vHPC_baseline_lfp_trace'),
 ('MD_baseline_lfp_trace', 'mPFC_baseline_lfp_trace'),
 ('MD_baseline_lfp_trace', 'vHPC_baseline_lfp_trace'),
 ('mPFC_baseline_lfp_trace', 'vHPC_baseline_lfp_trace')]

- Calculating the granger

In [ ]:
for region_1, region_2 in trial_brain_region_pairs:
    region_1_trimmed = region_1.split("_")[0]
    region_2_trimmed = region_2.split("_")[0]
    pair_base_name = "{}_{}".format(region_1_trimmed, region_2_trimmed)
    print(pair_base_name)
    try:
        multitaper_col = "{}_trial_multitaper".format(pair_base_name)
        # BASIC_LFP_TRACES_DF[multitaper_col] = BASIC_LFP_TRACES_DF.apply(lambda x: Multitaper(time_series=np.array([x[region_1],x[region_2]]).T, sampling_frequency=RESAMPLE_RATE, time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT), axis=1)
        BASIC_LFP_TRACES_DF[multitaper_col] = BASIC_LFP_TRACES_DF.apply(lambda x: Multitaper(time_series=np.array([x[region_1],x[region_2]]).T, sampling_frequency=RESAMPLE_RATE, time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, time_window_duration=TIME_WINDOW_DURATION, time_window_step=TIME_WINDOW_STEP), axis=1)
        print("\tcalculating connectivity")
        connectivity_col = "{}_trial_connectivity".format(pair_base_name)
        BASIC_LFP_TRACES_DF[connectivity_col] = BASIC_LFP_TRACES_DF[multitaper_col].apply(lambda x: Connectivity.from_multitaper(x))
        
        BASIC_LFP_TRACES_DF["{}_trial_frequencies".format(pair_base_name)] = BASIC_LFP_TRACES_DF[connectivity_col].apply(lambda x: x.frequencies)
        
        print("\t{}_{}_granger".format(region_1_trimmed, region_2_trimmed))
        BASIC_LFP_TRACES_DF["{}_{}_split_windows_granger".format(region_1_trimmed, region_2_trimmed)] = BASIC_LFP_TRACES_DF[connectivity_col].apply(lambda x: x.pairwise_spectral_granger_prediction()[:,:,0,1])

        BASIC_LFP_TRACES_DF["{}_{}_window_mean_granger".format(region_1_trimmed, region_2_trimmed)] = BASIC_LFP_TRACES_DF["{}_{}_split_windows_granger".format(region_1_trimmed, region_2_trimmed)].apply(lambda x: np.nanmean(x, axis=0))

        print("\t{}_{}_granger".format(region_2_trimmed, region_1_trimmed))
        BASIC_LFP_TRACES_DF["{}_{}_split_windows_granger".format(region_2_trimmed, region_1_trimmed)] = BASIC_LFP_TRACES_DF[connectivity_col].apply(lambda x: x.pairwise_spectral_granger_prediction()[:,:,1,0])

        BASIC_LFP_TRACES_DF["{}_{}_window_mean_granger".format(region_2_trimmed, region_1_trimmed)] = BASIC_LFP_TRACES_DF["{}_{}_split_windows_granger".format(region_2_trimmed, region_1_trimmed)].apply(lambda x: np.nanmean(x, axis=0))
        
        BASIC_LFP_TRACES_DF = BASIC_LFP_TRACES_DF.drop(columns=[connectivity_col], errors="ignore")
        BASIC_LFP_TRACES_DF.to_pickle("./proc/per_trial_granger.pkl")
    except Exception as e: 
        print(e)

BLA_LH
	calculating connectivity
	BLA_LH_granger


Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 7 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged


	LH_BLA_granger


Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 7 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged


BLA_MD
	calculating connectivity
	BLA_MD_granger


...

Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 17 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 17 of 19 converged
Maximum iterations reached. 3 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 0 of 19 converged


MD_mPFC
	calculating connectivity
	MD_mPFC_granger


Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 6 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 18 of 19 converged


	mPFC_MD_granger


Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 18 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 6 of 19 converged
Maximum iterations reached. 0 of 19 converged
Maximum iterations reached. 0 of 19 converged


In [ ]:
BASIC_LFP_TRACES_DF["BLA_LH_trial_frequencies"].iloc[0]

In [ ]:
BASIC_LFP_TRACES_DF

In [ ]:
GROUPINGS = "trial_outcome"

In [ ]:
all_frequencies = BASIC_LFP_TRACES_DF["BLA_LH_trial_frequencies"].iloc[0]

In [ ]:
FREQ_MAX

In [ ]:
low_freq = 0
high_freq = 100

for region_1, region_2 in trial_brain_region_pairs:
    region_1_trimmed = region_1.split("_")[0]
    region_2_trimmed = region_2.split("_")[0]
    
    
    fig, ax = plt.subplots()
    plt.title("granger of Z-scored LFP in {} to {}".format(region_1_trimmed, region_2_trimmed))
    plt.xlim(low_freq, high_freq)
    plt.ylim(0,1)

    ax.axvspan(4, 12, alpha=0.1, color='yellow', label="theta")
    ax.axvspan(12, 30, alpha=0.1, color='cyan', label="beta")
    ax.axvspan(30, 90, alpha=0.1, color='magenta', label="gamma")
    
    granger_col = "{}_{}_window_mean_granger".format(region_1_trimmed, region_2_trimmed)
    grouped_all_trials_df = BASIC_LFP_TRACES_DF.groupby([GROUPINGS]).agg({granger_col: lambda x: np.vstack(x.tolist())}).reset_index()
    # grouped_all_trials_df = grouped_all_trials_df[grouped_all_trials_df["trial_or_baseline"] == "trial"]
    grouped_all_trials_df["mean_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanmean(np.vstack(x), axis=0))
    grouped_all_trials_df["std_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanstd(np.vstack(x), axis=0))
    grouped_all_trials_df["n_trials"] = grouped_all_trials_df[granger_col].apply(lambda x: np.sum(~np.isnan(x), axis=0))
    grouped_all_trials_df["sem_granger"] = grouped_all_trials_df.apply(lambda x: x["std_granger"] / np.sqrt(x["n_trials"]), axis=1)
    for index, row in grouped_all_trials_df.iterrows():
        try:

            ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
            label="{}".format(row[GROUPINGS]), color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
            linewidth=3)

    
    
            plt.fill_between(all_frequencies, \
            row["mean_granger"] - row["sem_granger"], row["mean_granger"] + row["sem_granger"], alpha=0.1,
            color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]])
        except Exception as e: 
            print(e)

    plt.legend(ncol=3, loc="lower center")

    plt.savefig("./proc/{}_{}_granger_{}_{}hz.png".format(region_1_trimmed, region_2_trimmed, low_freq, high_freq))
    plt.show()
    
    fig, ax = plt.subplots()
    plt.title("granger of Z-scored LFP in {} to {}".format(region_2_trimmed, region_1_trimmed))
    plt.xlim(low_freq, high_freq)
    plt.ylim(0,1)

    ax.axvspan(4, 12, alpha=0.1, color='yellow', label="theta")
    ax.axvspan(12, 30, alpha=0.1, color='cyan', label="beta")
    ax.axvspan(30, 90, alpha=0.1, color='magenta', label="gamma")
    
    granger_col = "{}_{}_window_mean_granger".format(region_2_trimmed, region_1_trimmed)
    grouped_all_trials_df = BASIC_LFP_TRACES_DF.groupby([GROUPINGS]).agg({granger_col: lambda x: np.vstack(x.tolist())}).reset_index()
    # grouped_all_trials_df = grouped_all_trials_df[grouped_all_trials_df["trial_or_baseline"] == "trial"]
    grouped_all_trials_df["mean_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanmean(np.vstack(x), axis=0))
    grouped_all_trials_df["std_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanstd(np.vstack(x), axis=0))
    grouped_all_trials_df["n_trials"] = grouped_all_trials_df[granger_col].apply(lambda x: np.sum(~np.isnan(x), axis=0))
    grouped_all_trials_df["sem_granger"] = grouped_all_trials_df.apply(lambda x: x["std_granger"] / np.sqrt(x["n_trials"]), axis=1)
    for index, row in grouped_all_trials_df.iterrows():
        try:

            ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
            label="{}".format(row[GROUPINGS]), color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
            linewidth=3)

    
    
            plt.fill_between(all_frequencies, \
            row["mean_granger"] - row["sem_granger"], row["mean_granger"] + row["sem_granger"], alpha=0.1,
            color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]])
        except Exception as e: 
            print(e)
    
    plt.legend(ncol=3, loc="lower center")

    plt.savefig("./proc/{}_{}_granger_{}_{}hz.png".format(region_2_trimmed, region_1_trimmed, low_freq, high_freq))

    plt.show()
    
    


In [ ]:
all_frequencies[4:13]

In [ ]:
FREQUENCY_BANDS = (4,13)

In [ ]:
BAND_TO_FREQUENCY = {"theta": (4,13), "beta": (12,31), "gamma": (30, 91)}

In [ ]:
BAND_TO_ALL_COL = {"theta": [], "beta": [], "gamma": []}

In [ ]:
BASIC_LFP_TRACES_DF.columns

In [ ]:
all_granger_col = [col for col in BASIC_LFP_TRACES_DF.columns if "window_mean_granger" in col]


In [ ]:
all_granger_col

In [ ]:
for band, frequency in BAND_TO_FREQUENCY.items():
    for col in all_granger_col:
        BAND_COL = "{}_averaged_{}_granger".format("_".join(col.split("_")[:2]), band)
        BAND_TO_ALL_COL[band].append(BAND_COL)
        print(BAND_COL)
        print(frequency[0], frequency[1])
        BASIC_LFP_TRACES_DF[BAND_COL] = BASIC_LFP_TRACES_DF[col].apply(lambda x: np.nanmean(x[frequency[0]:frequency[1]]))    
        # BASIC_LFP_TRACES_DF[BAND_COL] = BASIC_LFP_TRACES_DF[col].apply(lambda x: x[frequency[0]:frequency[1]])    


In [ ]:
BASIC_LFP_TRACES_DF["BLA_LH_averaged_theta_granger"].iloc[0]

In [ ]:
BASIC_LFP_TRACES_DF[col].iloc[0].shape

In [ ]:
BASIC_LFP_TRACES_DF[GROUPINGS].unique()

In [ ]:
region_pair_to_outcome_to_granger = {k: defaultdict(nested_dict) for k,v in BAND_TO_FREQUENCY.items()}

for band, frequency in BAND_TO_FREQUENCY.items():
    for outcome in BASIC_LFP_TRACES_DF[GROUPINGS].unique():
        outcome_df = BASIC_LFP_TRACES_DF[BASIC_LFP_TRACES_DF[GROUPINGS] == outcome].copy()
        for band_col in set(BAND_TO_ALL_COL[band]):
            region_pair_to_outcome_to_granger[band]["_".join(band_col.split("_")[:-3])][outcome]["mean"] = outcome_df[band_col].mean() 
            region_pair_to_outcome_to_granger[band]["_".join(band_col.split("_")[:-3])][outcome]["sem"] = outcome_df[band_col].sem() 




In [ ]:
BAND_TO_ALL_COL[band]

In [ ]:
import matplotlib

In [ ]:
font = {'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [ ]:
for band, frequency in BAND_TO_FREQUENCY.items(): 
    # Convert the nested dictionary to a DataFrame
    data = []
    for group_name, group_data in region_pair_to_outcome_to_granger[band].items():
        for bar_name, bar_dict in group_data.items():
            data.append({"Group": group_name, "Bar": bar_name, "granger": bar_dict["mean"], "sem": bar_dict["sem"]})
    df = pd.DataFrame(data).sort_values(by=["Group", "Bar"])
    df["color"] = df["Bar"].map(BASELINE_OUTCOME_TO_COLOR)    
    # Create barplot
    ax = sns.barplot(x='Group', y='granger', hue='Bar', data=df, palette=df["color"], ci=None)
    
    # Adding error bars
    groups = df['Group'].unique()
    bars_per_group = df['Bar'].nunique()
    bar_width = 0.8 / bars_per_group  # the width of the bars
    x_positions = []  # this will store the x positions for the error bars

    for i, group in enumerate(groups):
        group_data = df[df['Group'] == group]  # filter the dataframe for the specific group
        num_bars = group_data.shape[0]  # get the number of bars for this group
        group_positions = np.linspace(i - bar_width*(num_bars-1)/2, i + bar_width*(num_bars-1)/2, num_bars)
        x_positions.extend(group_positions)  # add these positions to the list

        for pos, (idx, row) in zip(group_positions, group_data.iterrows()):
            height = row['granger']
            error = row['sem']
            plt.errorbar(pos, height, yerr=error, color='black', capsize=3, fmt='none', zorder=10)
    
    plt.xticks(rotation=90)
    plt.xlabel("Brain region pairs", fontsize=20)
    plt.ylabel("{} granger".format(band.title()), fontsize=20)
    plt.legend(title="Trial Conditions", loc="lower left", ncol=4)
    plt.title("{} granger".format(band.title()), fontsize=20)
    plt.tight_layout()
    plt.grid()
    plt.ylim(0,1)
    ax.get_legend().remove()

    plt.savefig("./proc/granger/all_zscored_{}_lfp_power_granger.png".format(band))
    # Show the plot
    plt.show()

In [ ]:
trial_outcome_pairs = ("win", "lose"), ("win", "rewarded"), ("lose", "omission")

In [ ]:
for band, frequency in BAND_TO_FREQUENCY.items():
    print(band)

In [ ]:
BASIC_LFP_TRACES_DF.columns

In [ ]:
BASIC_LFP_TRACES_DF["trial_outcome"].unique()

In [ ]:
region_to_mannwhitneyu = defaultdict(dict)
region_to_mannwhitneyu = []

for band, frequency in BAND_TO_FREQUENCY.items():
    print(band)
    for region_col in [col for col in BASIC_LFP_TRACES_DF.columns if band in col]:
        print(region_col)  
        for outcome_pair in trial_outcome_pairs:
            outcome_1_granger = BASIC_LFP_TRACES_DF[BASIC_LFP_TRACES_DF["trial_outcome"] == outcome_pair[0]][region_col].dropna()
            outcome_2_granger = BASIC_LFP_TRACES_DF[BASIC_LFP_TRACES_DF["trial_outcome"] == outcome_pair[1]][region_col].dropna()
            statistic, p_value = mannwhitneyu(outcome_1_granger, outcome_2_granger, alternative='two-sided')
            region_to_mannwhitneyu.append({"band": band, "region": region_col, "p_value": p_value, "statistic": statistic, "outcome": outcome_pair})
            
            # region_to_mannwhitneyu[region_col][outcome_pair] = p_value
            # print(region_col)
    
    
    # for outcome in BASIC_LFP_TRACES_DF[GROUPINGS].unique():
    #     outcome_df = BASIC_LFP_TRACES_DF[BASIC_LFP_TRACES_DF[GROUPINGS] == outcome].copy()
    #     for band_col in BAND_TO_ALL_COL[band]:
    #         region_pair_to_outcome_to_granger[band]["_".join(band_col.split("_")[:-3])][outcome]["mean"] = outcome_df[band_col].mean() 
    #         region_pair_to_outcome_to_granger[band]["_".join(band_col.split("_")[:-3])][outcome]["sem"] = outcome_df[band_col].sem() 




In [ ]:
granger_mannwhitneyu = pd.DataFrame(region_to_mannwhitneyu)

In [ ]:
granger_mannwhitneyu[granger_mannwhitneyu["p_value"] <= 0.001/3].sort_values(["p_value", "region"], ascending=True)

In [ ]:
BASIC_LFP_TRACES_DF.to_pickle("./proc/per_trial_granger.pkl")

In [ ]:
raise ValueError()

In [ ]:
GROUPINGS

In [ ]:
BASIC_LFP_TRACES_DF["BLA_LH_split_windows_granger"].iloc[0].shape

In [ ]:
low_freq = 30
high_freq = 90

for region_1, region_2 in trial_brain_region_pairs:
    region_1_trimmed = region_1.split("_")[0]
    region_2_trimmed = region_2.split("_")[0]
    
    
    fig, ax = plt.subplots()
    plt.title("granger of Z-scored LFP in {} to {}".format(region_1_trimmed, region_2_trimmed))
    plt.xlim(low_freq, high_freq)
    # plt.ylim(0,1)
    
    granger_col = "{}_{}_window_mean_granger".format(region_1_trimmed, region_2_trimmed)
    grouped_all_trials_df = BASIC_LFP_TRACES_DF.groupby([GROUPINGS]).agg({granger_col: lambda x: np.vstack(x.tolist())}).reset_index()
    # grouped_all_trials_df = grouped_all_trials_df[grouped_all_trials_df["trial_or_baseline"] == "trial"]
    grouped_all_trials_df["mean_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanmean(np.vstack(x), axis=0))
    grouped_all_trials_df["std_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanstd(np.vstack(x), axis=0))
    grouped_all_trials_df["n_trials"] = grouped_all_trials_df[granger_col].apply(lambda x: np.sum(~np.isnan(x), axis=0))
    grouped_all_trials_df["sem_granger"] = grouped_all_trials_df.apply(lambda x: x["std_granger"] / np.sqrt(x["n_trials"]), axis=1)
    for index, row in grouped_all_trials_df.iterrows():
        try:

            ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
            label="{}".format(row[GROUPINGS]), color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
            linewidth=3)

    
    
            plt.fill_between(all_frequencies, \
            row["mean_granger"] - row["sem_granger"], row["mean_granger"] + row["sem_granger"], alpha=0.1,
            color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]])
        except Exception as e: 
            print(e)
    # plt.savefig("./proc/{}_{}_granger_{}_{}hz.png".format(region_1_trimmed, region_2_trimmed, low_freq, high_freq))
    plt.show()
    
    fig, ax = plt.subplots()
    plt.title("granger of Z-scored LFP in {} to {}".format(region_2_trimmed, region_1_trimmed))
    plt.xlim(low_freq, high_freq)
    # plt.ylim(0,1)


    
    granger_col = "{}_{}_window_mean_granger".format(region_2_trimmed, region_1_trimmed)
    grouped_all_trials_df = BASIC_LFP_TRACES_DF.groupby([GROUPINGS]).agg({granger_col: lambda x: np.vstack(x.tolist())}).reset_index()
    # grouped_all_trials_df = grouped_all_trials_df[grouped_all_trials_df["trial_or_baseline"] == "trial"]
    grouped_all_trials_df["mean_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanmean(np.vstack(x), axis=0))
    grouped_all_trials_df["std_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanstd(np.vstack(x), axis=0))
    grouped_all_trials_df["n_trials"] = grouped_all_trials_df[granger_col].apply(lambda x: np.sum(~np.isnan(x), axis=0))
    grouped_all_trials_df["sem_granger"] = grouped_all_trials_df.apply(lambda x: x["std_granger"] / np.sqrt(x["n_trials"]), axis=1)
    for index, row in grouped_all_trials_df.iterrows():
        try:

            ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
            label="{}".format(row[GROUPINGS]), color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
            linewidth=3)

    
    
            plt.fill_between(all_frequencies, \
            row["mean_granger"] - row["sem_granger"], row["mean_granger"] + row["sem_granger"], alpha=0.1,
            color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]])
        except Exception as e: 
            print(e)
    # plt.savefig("./proc/{}_{}_granger_{}_{}hz.png".format(region_2_trimmed, region_1_trimmed, low_freq, high_freq))

    plt.show()
    
    


In [ ]:
raise ValueError()

In [ ]:
grouped_all_trials_df

In [ ]:
pair_base_name

In [ ]:
low_freq =30
high_freq = 90
for pair_base_name in all_pair_base_name:
    print(pair_base_name)
    
    fig, ax = plt.subplots()
    plt.title("granger of Z-scored LFP in {}".format(pair_base_name))
    plt.xlabel("Frequency")
    plt.ylabel("granger of Z-scored LFP")
    plt.xlim(low_freq, high_freq)
    plt.xticks(np.arange(low_freq, high_freq+1, 5))
    plt.yticks(np.arange(0, 1, 0.1))
    plt.ylim(0,1)
    plt.grid()

    for trial_or_baseline in TRIAL_AND_BASELINE:

        granger_col = "{}_{}_granger_magnitude".format(pair_base_name, trial_or_baseline)
        grouped_all_trials_df = channel_map_and_all_trials_df.groupby([GROUPINGS]).agg({granger_col: lambda x: np.vstack(x.tolist())}).reset_index()
        # grouped_all_trials_df = grouped_all_trials_df[grouped_all_trials_df["trial_or_baseline"] == "trial"]
        grouped_all_trials_df["mean_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanmean(np.vstack(x), axis=0))
        grouped_all_trials_df["std_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanstd(np.vstack(x), axis=0))
        grouped_all_trials_df["n_trials"] = grouped_all_trials_df[granger_col].apply(lambda x: np.sum(~np.isnan(x), axis=0))
        grouped_all_trials_df["sem_granger"] = grouped_all_trials_df.apply(lambda x: x["std_granger"] / np.sqrt(x["n_trials"]), axis=1)
        for index, row in grouped_all_trials_df.iterrows():
            try:
                if trial_or_baseline == "trial":
                    ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
                label="{} {}".format(row[GROUPINGS], trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                else:
                    ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
                color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                

    
                plt.fill_between(all_frequencies, \
                row["mean_granger"] - row["sem_granger"], row["mean_granger"] + row["sem_granger"], alpha=0.1,
                color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]])
            except Exception as e: 
                print(e)


    
    # plt.savefig("./proc/granger/{}_{}hz_{}_granger_of_zscored_lfp.png".format(low_freq, high_freq, pair_base_name))
    plt.show()
    
    
    

    

In [ ]:
raise ValueError()

In [ ]:
BASIC_LFP_TRACES_DF

In [ ]:
BASIC_LFP_TRACES_DF["BLA_trial_lfp_trace"]

In [ ]:
BASIC_LFP_TRACES_DF

In [ ]:
example = BASIC_LFP_TRACES_DF["BLA_LH_trial_connectivity"].iloc[0].pairwise_spectral_granger_prediction()

In [ ]:
example = BASIC_LFP_TRACES_DF["BLA_LH_trial_connectivity"].iloc[0]

In [ ]:
example.shape

In [ ]:
example[:,:100][0][1][0]

In [ ]:
example[:,:100][0][4][1]

In [ ]:
example[:,:100,1,0].mean(axis=0)

In [ ]:
example[:,:100,0,1].mean(axis=0)

In [ ]:
example.pairwise_spectral_granger_prediction()

In [ ]:
example.shape

In [ ]:
low_freq =30
high_freq = 90
for pair_base_name in all_pair_base_name:
    print(pair_base_name)
    
    fig, ax = plt.subplots()
    plt.title("granger of Z-scored LFP in {}".format(pair_base_name))
    plt.xlabel("Frequency")
    plt.ylabel("granger of Z-scored LFP")
    plt.xlim(low_freq, high_freq)
    plt.xticks(np.arange(low_freq, high_freq+1, 5))
    plt.yticks(np.arange(0, 1, 0.1))
    plt.ylim(0,1)
    plt.grid()

    for trial_or_baseline in TRIAL_AND_BASELINE:

        granger_col = "{}_{}_granger_magnitude".format(pair_base_name, trial_or_baseline)
        grouped_all_trials_df = channel_map_and_all_trials_df.groupby([GROUPINGS]).agg({granger_col: lambda x: np.vstack(x.tolist())}).reset_index()
        # grouped_all_trials_df = grouped_all_trials_df[grouped_all_trials_df["trial_or_baseline"] == "trial"]
        grouped_all_trials_df["mean_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanmean(np.vstack(x), axis=0))
        grouped_all_trials_df["std_granger"] = grouped_all_trials_df[granger_col].apply(lambda x: np.nanstd(np.vstack(x), axis=0))
        grouped_all_trials_df["n_trials"] = grouped_all_trials_df[granger_col].apply(lambda x: np.sum(~np.isnan(x), axis=0))
        grouped_all_trials_df["sem_granger"] = grouped_all_trials_df.apply(lambda x: x["std_granger"] / np.sqrt(x["n_trials"]), axis=1)
        for index, row in grouped_all_trials_df.iterrows():
            try:
                if trial_or_baseline == "trial":
                    ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
                label="{} {}".format(row[GROUPINGS], trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                else:
                    ax = sns.lineplot(x=all_frequencies, y=row["mean_granger"], \
                color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]],
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                

    
                plt.fill_between(all_frequencies, \
                row["mean_granger"] - row["sem_granger"], row["mean_granger"] + row["sem_granger"], alpha=0.1,
                color=BASELINE_OUTCOME_TO_COLOR[row[GROUPINGS]])
            except Exception as e: 
                print(e)


    
    # plt.savefig("./proc/granger/{}_{}hz_{}_granger_of_zscored_lfp.png".format(low_freq, high_freq, pair_base_name))
    plt.show()
    
    
    

    

In [ ]:
raise ValueError()

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [ ]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

## Calculating average power per band

In [ ]:
for col in BASIC_LFP_TRACES_DF.columns:
    print(col)

In [ ]:
chunk_averaged_power_columns = [col for col in BASIC_LFP_TRACES_DF.columns if "trial_lfp_trace" in col]

In [ ]:
chunk_averaged_power_columns

In [ ]:
for region_1 in chunk_averaged_power_columns:
    for region_2 in chunk_averaged_power_columns:
        

In [ ]:
from itertools import permutations 

In [ ]:
BASIC_LFP_TRACES_DF.columns

In [ ]:
np.array([BASIC_LFP_TRACES_DF.iloc[0][region_1], BASIC_LFP_TRACES_DF.iloc[0][region_2]]).T.shape

In [ ]:
for region_1, region_2 in permutations(chunk_averaged_power_columns, 2):
    pair_base_name = "{}_{}".format(region_1.split("_")[0], region_2.split("_")[0],)
    print(pair_base_name)
    try:
        
        # granger_value = grangercausalitytests(BASIC_LFP_TRACES_DF[[region_1, region_2]], maxlag=[3])
        BASIC_LFP_TRACES_DF["{}_granger".format(pair_base_name)] = BASIC_LFP_TRACES_DF.apply(lambda row: get_single_granger_causality(arr1=row[region_1], arr2=row[region_2]), axis=1)
        print()
    except Exception as e: 
        print(e)
    break

In [ ]:
BASIC_LFP_TRACES_DF

In [ ]:
for region_1, region_2 in permutations(chunk_averaged_power_columns, 2):
    pair_base_name = "{}_{}".format(region_1.strip("trace").strip("_"), region_2.strip("trace").strip("_"))
    print(pair_base_name)
    try:
        
        # granger_value = grangercausalitytests(BASIC_LFP_TRACES_DF[[region_1, region_2]], maxlag=[3])
        BASIC_LFP_TRACES_DF["{}_granger".format(pair_base_name)] = BASIC_LFP_TRACES_DF.apply(lambda row: grangercausalitytests(np.array([row[region_1], row[region_2]]).T, maxlag=1), axis=1)
        print()
    except Exception as e: 
        print(e)
    break

## Power correlation between brain regions calculation

- Combining the trial/baseline and outcome label for coloring

In [ ]:
BASIC_LFP_TRACES_DF["outcome_and_trial_or_baseline"] = BASIC_LFP_TRACES_DF.apply(lambda x: "_".join([x["trial_outcome"], x["trial_or_baseline"]]), axis=1)

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests



In [ ]:
BASIC_LFP_TRACES_DF.columns

In [ ]:
trace_columns = [col for col in BASIC_LFP_TRACES_DF.columns if "trace" in col]

In [ ]:
trace_columns

In [ ]:
brain_region_pairs = generate_pairs(trace_columns)

In [ ]:
brain_region_pairs

In [ ]:
grangercausalitytests(df[['column1', 'column2']], maxlag=[3])

In [ ]:
BASIC_LFP_TRACES_DF[region_1].iloc[0]

In [ ]:
for region_1, region_2 in brain_region_pairs:
    pair_base_name = "{}_{}".format(region_1.strip("trace").strip("_"), region_2.strip("trace").strip("_"))
    print(pair_base_name)
    try:
        
        # granger_value = grangercausalitytests(BASIC_LFP_TRACES_DF[[region_1, region_2]], maxlag=[3])
        BASIC_LFP_TRACES_DF["{}_granger".format(pair_base_name)] = BASIC_LFP_TRACES_DF.apply(lambda row: grangercausalitytests(np.array([row[region_1], row[region_2]]).T, maxlag=[3]), axis=1)
        print()
    except Exception as e: 
        print(e)
    break

In [ ]:
BASIC_LFP_TRACES_DF[""]

In [ ]:
BASIC_LFP_TRACES_DF

In [ ]:
granger_value

- Filtering out the outliers

In [ ]:
band_to_power_correlation = defaultdict(dict)
for band in ALL_BANDS:
    # Getting all the pairs of brain regions
    band_averaged_columns = [col for col in BASIC_LFP_TRACES_DF.columns if "averaged_{}".format(band) in col]
    band_to_power_correlation[band]["brain_region_pairs"] = generate_pairs(band_averaged_columns)
    print(band_to_power_correlation[band]["brain_region_pairs"])

    # Removing rows that are outliers
    filtered_df = BASIC_LFP_TRACES_DF.copy()
    
    for col in band_averaged_columns:
        # filtered_df = filtered_df[filtered_df[col] <= 3]
        # Assuming data is a 1D numpy array
        Q1 = np.percentile(filtered_df[col], 25)
        Q3 = np.percentile(filtered_df[col], 75)
        IQR = Q3 - Q1
        band_to_power_correlation[band]["outlier_removed_df"] = filtered_df[(filtered_df[col] >= Q1 - 1.5 * IQR) & (filtered_df[col] <= Q3 + 1.5 * IQR)]


    
    # Getting the mean and standard deviation
    

In [ ]:
band_to_power_correlation = defaultdict(dict)
for band in ALL_BANDS:
    # Getting all the pairs of brain regions
    band_averaged_columns = [col for col in BASIC_LFP_TRACES_DF.columns if "averaged_{}".format(band) in col]
    band_to_power_correlation[band]["brain_region_pairs"] = generate_pairs(band_averaged_columns)
    print(band_to_power_correlation[band]["brain_region_pairs"])

    # Removing rows that are outliers
    filtered_df = BASIC_LFP_TRACES_DF.copy()
    
    for col in band_averaged_columns:
        # filtered_df = filtered_df[filtered_df[col] <= 3]
        # Assuming data is a 1D numpy array
        Q1 = np.percentile(filtered_df[col], 25)
        Q3 = np.percentile(filtered_df[col], 75)
        IQR = Q3 - Q1
        filtered_df = filtered_df[(filtered_df[col] >= Q1 - 1.5 * IQR) & (filtered_df[col] <= Q3 + 1.5 * IQR)]
    band_to_power_correlation[band]["outlier_removed_df"] = filtered_df

In [ ]:
BASIC_LFP_TRACES_DF.shape

In [ ]:
band_to_power_correlation[band]["outlier_removed_df"].shape

- Plotting all of the conditions

In [ ]:
for band in ALL_BANDS:
    for region_1, region_2 in band_to_power_correlation[band]["brain_region_pairs"]:
        region_1_basename = region_1.split("_")[0]
        region_2_basename = region_2.split("_")[0]
        x = band_to_power_correlation[band]["outlier_removed_df"][region_1]
        y = band_to_power_correlation[band]["outlier_removed_df"][region_2]
        
        # Perform linear regression to get the slope, intercept and r-value (correlation coefficient)
        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        
        # Create a line of best fit using the slope and intercept
        line = slope * x + intercept
        
        # Create scatter plot
        sns.scatterplot(x=x, y=y, data=band_to_power_correlation[band]["outlier_removed_df"], hue='outcome_and_trial_or_baseline', palette=BASELINE_OUTCOME_TO_COLOR)
        
        # Plot line of best fit
        plt.plot(x, line, color='red')
        
        # Add R² value to the plot
        plt.text(0.1, 0.9, f'R = {r_value:.2f}', transform=plt.gca().transAxes)
        
        # Add labels and legend
        plt.title("Power correlation of Z-scored {} band LFP: {} and {}".format(band, region_2_basename, region_1_basename))
        plt.xlabel('{} {} power of Z-scored LFP'.format(band, region_1_basename))
        plt.ylabel('{} {} power of Z-scored LFP'.format(band, region_2_basename))
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.savefig("./proc/power_correlation/zscored/{}/all_condition_{}_{}_power_correlation_of_zscored_{}_lfp.png".format(band, region_1_basename, region_2_basename, band))
        # Display the plot
        plt.show()




In [ ]:
raise ValueError()

In [ ]:
BASIC_LFP_TRACES_DF = filtered_df

In [ ]:
BASIC_LFP_TRACES_DF["trial_outcome"].unique()

In [ ]:
BASIC_LFP_TRACES_DF["trial_or_baseline"]

In [ ]:
for band in ALL_BANDS:
    band_df = band_to_power_correlation[band]["outlier_removed_df"]
    band_to_power_correlation[band]["region_pair_to_outcome_to_r2"] = defaultdict(nested_dict)
    for outcome in band_df["trial_outcome"].unique():
        outcome_df = band_df[band_df["trial_outcome"] == outcome]
        for region_1, region_2 in brain_region_pairs:
            region_1_basename = region_1.split("_")[0]
            region_2_basename = region_2.split("_")[0]
            
            x = outcome_df[region_1]
            y = outcome_df[region_2]
            
            # Perform linear regression to get the slope, intercept and r-value (correlation coefficient)
            slope, intercept, r_value, p_value, std_err = linregress(x, y)
            # Square the r value to get the r squared value
            r2_value = r_value**2
            band_to_power_correlation[band]["region_pair_to_outcome_to_r2"]["{}_{}".format(region_1.split("_")[0], region_2.split("_")[0])][outcome]["r"] = r_value
            band_to_power_correlation[band]["region_pair_to_outcome_to_r2"]["{}_{}".format(region_1.split("_")[0], region_2.split("_")[0])][outcome]["std"] = std_err
            
            # Create a line of best fit using the slope and intercept
            line = slope * x + intercept
            
            # Create scatter plot
            sns.scatterplot(x=x, y=y, data=outcome_df, hue='outcome_and_trial_or_baseline', palette=BASELINE_OUTCOME_TO_COLOR, style='outcome_and_trial_or_baseline', markers=['^', 'o'])
            
            # Plot line of best fit
            plt.plot(x, line, color='red')
            
            # Add R² value to the plot
            plt.text(0.1, 0.9, f'R = {r_value:.2f}', transform=plt.gca().transAxes)
            
            # Add labels and legend
            plt.title("Power Correlation of Z-scored {} LFP: {} and {}".format(band, region_2_basename, region_1_basename))
            plt.xlabel('{} {} Power of Z-scored LFP'.format(region_1_basename, band))
            plt.ylabel('{} {} Power of Z-scored LFP'.format(region_2_basename, band))
            plt.legend(loc="lower right")
            plt.tight_layout()
            plt.savefig("./proc/power_correlation/zscored/{}/{}_{}_{}_power_correlation_of_zscored_{}_lfp.png".format(band, outcome, region_1_basename, region_2_basename, band))
            # Display the plot
            plt.show()

In [ ]:
for band in ALL_BANDS:
    # Convert the nested dictionary to a DataFrame
    data = []
    for group_name, group_data in band_to_power_correlation[band]['region_pair_to_outcome_to_r2'].items():
        for bar_name, bar_dict in group_data.items():
            data.append({"Group": group_name, "Bar": bar_name, "r": bar_dict["r"], "std": bar_dict["std"]})
    df = pd.DataFrame(data)
    
    # Create the bar plot using seaborn
    # sns.catplot(
    #     data=df, 
    #     x='Group', 
    #     y='r2', 
    #     hue='Bar', 
    #     kind='bar', 
    #     height=4, 
    #     aspect=2,
    #     legend=False,
    #     # yerr=df['std'].values,  # This line adds the SEM bars
    #     # capsize=0.1  # This line adds caps on the error bars
    # )
    
    # Create barplot
    ax = sns.barplot(x='Group', y='r', hue='Bar', data=df, ci=None)
    
    # Adding error bars
    groups = df['Group'].unique()
    bars_per_group = df['Bar'].nunique()
    bar_width = 0.8 / bars_per_group
    x_positions = []
    
    for i, group in enumerate(groups):
        num_bars = df[df['Group'] == group].shape[0]
        group_positions = np.linspace(i - bar_width*(num_bars-1)/2, i + bar_width*(num_bars-1)/2, num_bars)
        x_positions.extend(group_positions)
    
    for i, (r2, sem) in enumerate(zip(df['r'], df['std'])):
        plt.errorbar(x_positions[i], r2, yerr=sem, fmt='none', color='black', capsize=5)
    
    
    plt.xticks(rotation=90)
    plt.xlabel("Brain region pairs")
    plt.ylabel("Power correlation r")
    plt.legend(title="Trial Conditions")
    plt.title("{} Power correlations".format(band))
    plt.tight_layout()
    plt.grid()
    
    plt.savefig("./proc/power_correlation/zscored/all_zscored_{}_lfp_power_correlation.png".format(band))
    # Show the plot
    plt.show()

In [ ]:

# Convert the nested dictionary to a DataFrame
data = []
for group_name, group_data in region_pair_to_outcome_to_r2.items():
    for bar_name, bar_dict in group_data.items():
        data.append({"Group": group_name, "Bar": bar_name, "r": bar_dict["r"], "std": bar_dict["std"]})
df = pd.DataFrame(data)

# Create the bar plot using seaborn
# sns.catplot(
#     data=df, 
#     x='Group', 
#     y='r2', 
#     hue='Bar', 
#     kind='bar', 
#     height=4, 
#     aspect=2,
#     legend=False,
#     # yerr=df['std'].values,  # This line adds the SEM bars
#     # capsize=0.1  # This line adds caps on the error bars
# )

# Create barplot
ax = sns.barplot(x='Group', y='r', hue='Bar', data=df, ci=None)

# Adding error bars
groups = df['Group'].unique()
bars_per_group = df['Bar'].nunique()
bar_width = 0.8 / bars_per_group
x_positions = []

for i, group in enumerate(groups):
    num_bars = df[df['Group'] == group].shape[0]
    group_positions = np.linspace(i - bar_width*(num_bars-1)/2, i + bar_width*(num_bars-1)/2, num_bars)
    x_positions.extend(group_positions)

for i, (r2, sem) in enumerate(zip(df['r'], df['std'])):
    plt.errorbar(x_positions[i], r2, yerr=sem, fmt='none', color='black', capsize=5)


plt.xticks(rotation=90)
plt.xlabel("Brain region pairs")
plt.ylabel("Power correlation r")
plt.legend(title="Trial Conditions")
plt.title("Power correlations")
plt.tight_layout()
plt.grid()

plt.savefig("./proc/power_correlation/zscored/all_zscored_lfp_power_correlation.png")
# Show the plot
plt.show()

In [ ]:
bars

In [ ]:
import statannot
statannot.add_stat_annotation(
    ax,
    data=df,
    x=x,
    y=y,
    hue=hue,
    box_pairs=[
        (("Biscoe", "Male"), ("Torgersen", "Female")),
        (("Dream", "Male"), ("Dream", "Female")),
    ],
    test="t-test_ind",
    text_format="star",
    loc="outside",
)

In [ ]:
x_coords

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
groups = ['Group1', 'Group2', 'Group3']
values = [1.2, 2.3, 1.8]

# Dictionary holding the results of your statistical comparisons
# Keys are tuples indicating the groups being compared; values are the corresponding p-values
stats_dict = {('Group1', 'Group2'): 0.04, ('Group1', 'Group3'): 0.01, ('Group2', 'Group3'): 0.01}

# Thresholds for significance levels
alpha = 0.05  # usually 0.05
alpha_strong = 0.01  # example value

# Create bar plot
fig, ax = plt.subplots()
bars = plt.bar(groups, values, color=['blue', 'green', 'red'])

# Get the y-axis limits
bottom, top = ax.get_ylim()
y_range = top - bottom

# Retrieve x-coordinates of the bars
x_coords = [bar.get_x() + bar.get_width() / 2.0 for bar in bars]  # get_x() retrieves the left coordinate, we adjust by half the width to get the center
group_to_x_coord = {group: x_coord for group, x_coord in zip(groups, x_coords)}


# Significance bars
for i, (key, value) in enumerate(stats_dict.items()):
    # Significance level
    p = value
    if p < 0.001:
        sig_symbol = '***'
    elif p < 0.01:
        sig_symbol = '**'
    elif p < 0.05:
        sig_symbol = '*'
    else:
        continue
    
    # Columns corresponding to the datasets of interest
    x1 = key[0]
    x2 = key[1]
    # What level is this bar among the bars above the plot?
    level = len(stats_dict) - i
    # Plot the bar
    bar_height = (y_range * 0.15 * level) + top
    bar_tips = bar_height - (y_range * 0.02)
    plt.plot(
        [x1, x1, x2, x2],
        [bar_tips, bar_height, bar_height, bar_tips], lw=1, c='k'
    )
    
    text_height = bar_height + (y_range * 0.01)
    plt.text((group_to_x_coord[x1] + group_to_x_coord[x2]) * 0.5, text_height, sig_symbol, ha='center', va='bottom', c='k')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
groups = ['Group1', 'Group2', 'Group3']
values = [1.2, 2.3, 1.8]

# Dictionary holding the results of your statistical comparisons
# Keys are tuples indicating the groups being compared; values are the corresponding p-values
stats_dict = {('Group1', 'Group2'): 0.04, ('Group1', 'Group3'): 0.01, ('Group2', 'Group3'): 0.01}

# Thresholds for significance levels
alpha = 0.05  # usually 0.05
alpha_strong = 0.01  # example value

# Create bar plot
fig, ax = plt.subplots()
bars = plt.bar(groups, values, color=['blue', 'green', 'red'])

# Function to draw significance bars
def draw_significance_bar(ax, x1, x2, y, height, text, offset):
    ax.plot([x1, x1, x2, x2], [y + offset, y + height + offset, y + height + offset, y + offset], lw=1.5, c='black')
    ax.text((x1 + x2) * .5, y + height + offset, text, ha='center', va='bottom', color='black')

# Height of the small vertical ticks and offset for multiple comparisons
tick_height = 0.1
offset_step = 0.1  # How much to offset each additional line

# Track the current offset
current_offsets = {group: 0 for group in groups}

# Check each comparison
for comparison, p_value in stats_dict.items():
    # Determine the index of the groups
    index_1 = groups.index(comparison[0])
    index_2 = groups.index(comparison[1])

    # Set the positions of bars on X axis
    x1, x2 = bars[index_1].xy[0] + 0.4, bars[index_2].xy[0] + 0.4  # we use 0.4 to center the line on the bar
    max_height = max([bar.get_height() for bar in bars])
    y = values[index_1] if values[index_1] > values[index_2] else values[index_2]  # get the maximum value among the two groups
    
    # Choose the symbol to display depending on the p-value
    if p_value < alpha_strong:
        text = '**'  # or '***' for an even smaller p-value
    elif p_value < alpha:
        text = '*'
    else:
        continue  # Don't draw a bar if not significant

    # Determine the offset for this line
    offset = max(current_offsets[comparison[0]], current_offsets[comparison[1]])
    
    # Draw the significance bar
    draw_significance_bar(ax, x1, x2, y, tick_height, text, offset)

    # Update the current offset for these groups
    current_offsets[comparison[0]] = current_offsets[comparison[1]] = offset + offset_step

plt.show()